<a href="https://colab.research.google.com/github/nkrumahthis/alu-x-meta-stitches/blob/main/My_Copy_of_llama3_2_via_vertex_ai_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Image-to-text Inference with Llama 3.2 90B Vision Instruct Model

This notebook demonstrates how to perform inference on a llama3.2 90B Vision Model.

Make a copy of this file and do not directly edit it.

Install the required libraries

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform openai google

Imports

In [ ]:
import openai

from google.auth import default, transport
from google.cloud import storage

from google.oauth2 import service_account
from google.colab import auth

from PIL import Image
import requests

import io
from IPython.display import display

import base64

import re
import json

### Authenticate using service account credentials

A service account credential file will be shared with you at the beginning of the session. To set up authentication:

 Open the "sample_data" directory and create a new file called "cred.json". Copy the contents of the shared credential file into it.

1.   Click on the file icon to the left of your screen to access files
2.   Open the sample_date directory
3.   Upload the service account credentials json file shared with you
4.   If necessary, change path of KEY_FILE_PATH to match the uploaded file



In [ ]:
# Replace with the path to your shared JSON key file
KEY_FILE_PATH = './cred.json'

# Load credentials from the JSON key file
credentials = service_account.Credentials.from_service_account_file(
    KEY_FILE_PATH,
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)

# Authenticate using the service account credentials
auth.authenticate_user()

Refresh Credentials.
Creates credentials with a lifespan of 1hour.

In [ ]:
credentials, _ = default()
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

Set up project and model details

In [ ]:
LOCATION = "us-central1"
MAAS_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"
PROJECT_ID="studious-plate-436913-c1"
MODEL_ID="meta/llama-3.2-90b-vision-instruct-maas"
BUCKET_URI = "gs://metallama3bootcamp"

Likely unneccesary to run this, as the Storage Bucket already exists.
Create a Google Cloud Storage Bucket for the roject.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Set up the client, design the prompt, and run inference.

In [ ]:
openai_client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

Download the image

In [ ]:

# Specify your bucket and blob name
# blob_name = "pra_2020-11-18.png"
blob_name = "pra_2014-02-20.png"

prompt = """
given this image of a river, give me your answer in json like

```{"name": "Birim", "muddiness_score":1, "date": "2023-12-12"}```

on this criteria:
name: get it from the file name
date: get it from the file name
muddiness_score: give a muddiness_score based on how brown the river looks, on a score of 0 to 10. if the river looks bluish or greenish, give it a 0-4 score. if it looks brown, give it a high score 5-10.

file name:
""" + blob_name + """

**Answer:**
"""

# Set the maximum number of tokens to generate
max_tokens=250

print(prompt)


In [ ]:
def find_json(text):
  match = re.search(r'\{.*\}', text)
  if match:
    try:
      json_string = match.group(0)
      json_object = json.loads(json_string)
      return json_object
    except ValueError:
      return None
  else:
    return None

prompt_jsons = []

In [ ]:
# Replace with your bucket name
bucket_name = "metallama3bootcamp"

storage_client = storage.Client()

# Get the bucket
bucket = storage_client.bucket(bucket_name)

# List all blobs in the bucket
blobs = bucket.list_blobs()

# Create a list of blob names
blob_names = [blob.name for blob in blobs]

# Print the list of blob names
print(blob_names)


In [ ]:
for blob_name in blob_names:
  river_image_url = "gs://metallama3bootcamp/" + blob_name

  prompt = """
given this image of a river, give me your answer in json like

```{"name": "Birim", "muddiness_score":1, "date": "2023-12-12"}```

on this criteria:
name: get it from the file name
date: get it from the file name
muddiness_score: give a muddiness_score based on how brown the river looks, on a score of 0 to 10. if the river looks bluish or greenish, give it a 0-4 score. if it looks brown, give it a high score 5-10.

file name:
""" + blob_name + """

**Answer:**
"""

  print(prompt)

  # Download the image
  blob = bucket.blob(blob_name)

  if blob_name.split(".")[-1] != "png":
    continue

  # Read the image into memory
  image_data = blob.download_as_bytes()
  image = Image.open(io.BytesIO(image_data))

  response = openai_client.chat.completions.create(
      model=MODEL_ID,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "image_url": {
                          "url": river_image_url
                      },
                      "type": "image_url",
                  },
                  {"text": prompt, "type": "text"},
              ],
          },
      ],
      max_tokens=max_tokens,
  )

  display(image)
  print(response.choices[0].message.content)
  answer = response.choices[0].message.content



In [ ]:


json_object = find_json(answer)
print(json_object)